**Copyright 20201 Die Autoren der TF-Agenten.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SAC-Minitaurus mit der Actor-Learner-API

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Führen Sie in Google Colab aus</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a></td>
</table>


## Einführung

This example shows how to train a [Soft Actor Critic](https://arxiv.org/abs/1812.05905) agent on the [Minitaur](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py) environment.

Wenn Sie das [DQN Colab durchgearbeitet](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb) haben, sollte sich dies sehr vertraut anfühlen. Bemerkenswerte Änderungen umfassen:

- Ändern des Agenten von DQN zu SAC.
- Training auf Minitaur, einer viel komplexeren Umgebung als CartPole. Die Minitaur-Umgebung zielt darauf ab, einen Vierbeiner zu trainieren, um sich vorwärts zu bewegen.
- Verwenden der TF-Agents Actor-Learner-API für verteiltes Reinforcement-Lernen.

Die API unterstützt sowohl die verteilte Datenerfassung mithilfe eines Erfahrungswiedergabepuffers und eines variablen Containers (Parameterserver) als auch das verteilte Training auf mehrere Geräte. Die API ist sehr einfach und modular aufgebaut. Wir verwenden [Reverb](https://deepmind.com/research/open-source/Reverb) sowohl für den Wiedergabepuffer als auch für den variablen Container und die [TF DistributionStrategy-API](https://www.tensorflow.org/guide/distributed_training) für verteiltes Training auf GPUs und TPUs.

Wenn Sie die folgenden Abhängigkeiten nicht installiert haben, führen Sie Folgendes aus:

In [ ]:
!sudo apt-get install -y xvfb ffmpeg
!pip install 'gym==0.10.11'
!pip install 'imageio==2.4.0'
!pip install matplotlib
!pip install PILLOW
!pip install tf-agents[reverb]
!pip install 'pybullet==2.4.2'

## Konfiguration

Zuerst importieren wir die verschiedenen Tools, die wir benötigen.

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.experimental.train import actor
from tf_agents.experimental.train import learner
from tf_agents.experimental.train import triggers
from tf_agents.experimental.train.utils import spec_utils
from tf_agents.experimental.train.utils import strategy_utils
from tf_agents.experimental.train.utils import train_utils
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils

tempdir = tempfile.gettempdir()

## Hyperparameter

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## Umgebung

Umgebungen in RL stellen die Aufgabe oder das Problem dar, die wir lösen möchten. Standardumgebungen können in TF-Agenten mithilfe von `suites` einfach erstellt werden. Wir haben verschiedene `suites` zum Laden von Umgebungen aus Quellen wie OpenAI Gym, Atari, DM Control usw., denen ein String-Umgebungsname zugewiesen wurde.

Laden wir nun die Minituar-Umgebung aus der Pybullet-Suite.

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

In dieser Umgebung besteht das Ziel darin, dass der Agent eine Richtlinie trainiert, die den Minitaur-Roboter steuert und ihn so schnell wie möglich vorwärts bringt. Episoden dauern 1000 Schritte und die Rückkehr ist die Summe der Belohnungen während der gesamten Episode.

Let's look at the information the environment provides as an `observation` which the policy will use to generate `actions`.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Wie wir sehen können, ist die Beobachtung ziemlich komplex. Wir erhalten 28 Werte, die die Winkel, Geschwindigkeiten und Drehmomente für alle Motoren darstellen. Im Gegenzug erwartet die Umgebung 8 Werte für die Aktionen zwischen `[-1, 1]` . Dies sind die gewünschten Motorwinkel.

Normalerweise erstellen wir zwei Umgebungen: eine zum Sammeln von Daten während des Trainings und eine zum Auswerten. Die Umgebungen sind in reinem Python geschrieben und verwenden Numpy-Arrays, die von der Actor Learner-API direkt verwendet werden.

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## Vertriebsstrategie

Wir verwenden die DistributionStrategy-API, um die Ausführung der Zugschrittberechnung über mehrere Geräte wie mehrere GPUs oder TPUs mithilfe von Datenparallelität auszuführen. Der Zugschritt:

- Erhält eine Reihe von Trainingsdaten
- Teilt es auf die Geräte auf
- Berechnet den Vorwärtsschritt
- Aggregiert und berechnet die BEDEUTUNG des Verlusts
- Berechnet den Rückwärtsschritt und führt eine Aktualisierung der Gradientenvariablen durch

Mit der TF-Agents Learner API und der DistributionStrategy API können Sie ganz einfach zwischen dem Ausführen des Zugschritts auf GPUs (mit MirroredStrategy) und TPUs (mit TPUStrategy) wechseln, ohne die unten stehende Trainingslogik zu ändern.

### Aktivieren der GPU

Wenn Sie versuchen möchten, auf einer GPU ausgeführt zu werden, müssen Sie zuerst die GPUs für das Notebook aktivieren:

Navigieren Sie zu Bearbeiten → Notebook-Einstellungen, und wählen Sie in der Dropdown-Liste Hardwarebeschleuniger die Option GPU aus

### Eine Strategie auswählen

Verwenden Sie `strategy_utils` , um eine Strategie zu generieren. Übergeben Sie unter der Haube den Parameter:

- `use_gpu = False` gibt `tf.distribute.get_strategy()` , das die CPU verwendet
- `use_gpu = True` gibt `tf.distribute.MirroredStrategy()` , das alle GPUs verwendet, die für TensorFlow auf einem Computer sichtbar sind

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

Alle Variablen und Agenten müssen unter `strategy.scope()` , wie Sie unten sehen werden.

## Agent

Um einen SAC-Agenten zu erstellen, müssen wir zuerst die Netzwerke erstellen, die er trainieren wird. SAC ist ein Schauspieler-Kritiker-Agent, daher brauchen wir zwei Netzwerke.

Der Kritiker wird uns Wertschätzungen für `Q(s,a)` . Das heißt, es wird als Eingabe eine Beobachtung und eine Handlung erhalten, und es wird uns eine Schätzung geben, wie gut diese Handlung für den gegebenen Zustand war.


In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

Wir werden diese Kritiker verwenden , um einen Zug `actor` - Netzwerk , das uns Aktionen eine Beobachtung gegeben generieren können.

Das `ActorNetwork` Parameter für eine tanh-gequetschte [MultivariateNormalDiag-](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag) Verteilung voraus. Diese Verteilung wird dann unter Berücksichtigung der aktuellen Beobachtung abgetastet, wann immer wir Aktionen generieren müssen.

In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

Mit diesen Netzwerken können wir jetzt den Agenten instanziieren.


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## Puffer wiedergeben

Um den Überblick über die aus der Umgebung gesammelten Daten zu behalten, verwenden wir [Reverb](https://deepmind.com/research/open-source/Reverb) , ein effizientes, erweiterbares und benutzerfreundliches Wiedergabesystem von Deepmind. Es speichert Erfahrungsdaten, die von den Schauspielern gesammelt und vom Lernenden während des Trainings konsumiert wurden.

In diesem Lernprogramm ist dies weniger wichtig als `max_size` In einer verteilten Umgebung mit asynchroner Erfassung und Schulung möchten Sie wahrscheinlich mit `rate_limiters.SampleToInsertRatio` experimentieren und dabei ein samples_per_insert zwischen 2 und 1000 verwenden. Beispiel:

```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0))
```


In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

The replay buffer is constructed using specs describing the tensors that are to be stored, which can be obtained from the agent using `tf_agent.collect_data_spec`.

Da der SAC-Agent sowohl die aktuelle als auch die nächste Beobachtung benötigt, um den Verlust zu berechnen, setzen wir `sequence_length=2` .

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

Jetzt generieren wir einen TensorFlow-Datensatz aus dem Reverb-Wiedergabepuffer. Wir werden dies an den Lernenden weitergeben, um Erfahrungen für das Training zu sammeln.

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## Richtlinien

In TF-Agents, policies represent the standard notion of policies in RL: given a `time_step` produce an action or a distribution over actions. The main method is `policy_step = policy.step(time_step)` where `policy_step` is a named tuple `PolicyStep(action, state, info)`.  The `policy_step.action` is the `action` to be applied to the environment, `state` represents the state for stateful (RNN) policies and `info` may contain auxiliary information such as log probabilities of the actions.

Agenten enthalten zwei Richtlinien:

- `agent.policy` - Die Hauptrichtlinie, die für die Evaluierung und Bereitstellung verwendet wird.
- `agent.collect_policy` - Eine zweite Richtlinie, die für die Datenerfassung verwendet wird.

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

Richtlinien können unabhängig von Agenten erstellt werden. Verwenden Sie beispielsweise `tf_agents.policies.random_py_policy` , um eine Richtlinie zu erstellen, die zufällig eine Aktion für jeden Zeitschritt auswählt.

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## Schauspieler

Der Akteur verwaltet die Interaktionen zwischen einer Richtlinie und einer Umgebung.

- Die Actor-Komponenten enthalten eine Instanz der Umgebung (als `py_environment` ) und eine Kopie der Richtlinienvariablen.
- Jeder Actor-Worker führt eine Folge von Datenerfassungsschritten unter Berücksichtigung der lokalen Werte der Richtlinienvariablen aus.
- Variable updates are done explicitly using the variable container client instance in the training script before calling `actor.run()`.
- Die beobachtete Erfahrung wird in jedem Datenerfassungsschritt in den Wiedergabepuffer geschrieben.

Während die Akteure Datenerfassungsschritte ausführen, geben sie Trajektorien von (Status, Aktion, Belohnung) an den Beobachter weiter, der sie zwischenspeichert und in das Reverb-Wiedergabesystem schreibt.

Wir speichern Trajektorien für Frames [(t0, t1) (t1, t2) (t2, t3), ...], weil `stride_length=1` .

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

Wir erstellen einen Akteur mit der Zufallsrichtlinie und sammeln Erfahrungen, um den Wiedergabepuffer zu setzen.

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

Instanziieren Sie einen Schauspieler mit der Sammlungsrichtlinie, um während des Trainings mehr Erfahrungen zu sammeln.

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

Erstellen Sie einen Akteur, mit dem die Richtlinie während des Trainings bewertet wird. Wir übergeben `actor.eval_metrics(num_eval_episodes)` , um Metriken später zu protokollieren.

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## Lernende

Die Learner-Komponente enthält den Agenten und führt mithilfe von Erfahrungsdaten aus dem Wiedergabepuffer Gradientenschrittaktualisierungen an den Richtlinienvariablen durch. Nach einem oder mehreren Trainingsschritten kann der Lernende einen neuen Satz variabler Werte in den variablen Container verschieben.

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)

## Metriken und Auswertung

Wir haben den eval Actor oben `actor.eval_metrics` instanziiert, wodurch die am häufigsten verwendeten Metriken während der Richtlinienbewertung erstellt werden:

- Durchschnittliche Rendite. Die Rendite ist die Summe der Belohnungen, die beim Ausführen einer Richtlinie in einer Umgebung für eine Episode erzielt werden. In der Regel wird dies über einige Episoden gemittelt.
- Durchschnittliche Episodenlänge.

Wir führen den Akteur aus, um diese Metriken zu generieren.

In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

Überprüfen Sie das [Metrikmodul](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py) auf andere Standardimplementierungen verschiedener Metriken.

## Schulung des Agenten

Die Trainingsschleife umfasst sowohl das Sammeln von Daten aus der Umgebung als auch das Optimieren der Netzwerke des Agenten. Unterwegs werden wir gelegentlich die Richtlinien des Agenten bewerten, um festzustellen, wie es uns geht.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## Visualisierung


### Grundstücke

Wir können die durchschnittliche Rendite im Vergleich zu globalen Schritten darstellen, um die Leistung unseres Agenten zu sehen. In `Minitaur` basiert die Belohnungsfunktion darauf, wie weit der Minitaur in 1000 Schritten geht und den Energieverbrauch bestraft.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### Videos

Es ist hilfreich, die Leistung eines Agenten zu visualisieren, indem die Umgebung bei jedem Schritt gerendert wird. Bevor wir das tun, erstellen wir zunächst eine Funktion zum Einbetten von Videos in diese Spalte.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Der folgende Code veranschaulicht die Richtlinien des Agenten für einige Episoden:

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)